# Battery Analysis v2 사용 예제

이 노트북은 `battery_analysis_v2` 모듈의 주요 기능을 시연합니다.

## 주요 기능

1. **용량 열화 모델링** (`CapacityDegradationModel`)
2. **승인 수명 예측** (`ApprovalLifePredictor`)
3. **dV/dQ 분석** (Full-cell 시뮬레이션)

---

## 1. 환경 설정 및 Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# battery_analysis_v2 모듈 import
from battery_analysis_v2.core.life_prediction.empirical import (
    CapacityDegradationModel,
    ApprovalLifePredictor,
    ModelParameters,
    capacityfit
)

# 그래프 스타일 설정
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
print('✓ 모듈 import 완료')

## 2. 용량 열화 모델링 (CapacityDegradationModel)

경험적 용량 열화 모델을 사용하여 배터리 용량 예측을 수행합니다.

### 수식
```
capacity(cycle, T) = 1 - exp(a*T + b) * (cycle*fd)^b1 - exp(c*T + d) * (cycle*fd)^(e*T + f)
```

- 첫 번째 항: Calendar aging (저장 열화)
- 두 번째 항: Cycle aging (사이클 열화)

In [ ]:
# 예제 데이터 생성 (실제 실험 데이터로 대체 가능)
np.random.seed(42)

# 사이클 수
cycles = np.arange(0, 1000, 50)

# 온도 (K) - 45°C 시험
temperatures = np.full_like(cycles, 318.15, dtype=float)

# 실제 용량 데이터 (시뮬레이션)
# 기본 파라미터로 용량 곡선 생성 + 노이즈
true_capacity = capacityfit(
    (cycles, temperatures),
    a=0.03, b=-18, b1=0.7,
    c=2.3, d=-782, e=-0.28, f=96, fd=1
)
# 노이즈 추가
capacities = true_capacity + np.random.normal(0, 0.01, len(cycles))

# 데이터 시각화
plt.figure(figsize=(10, 5))
plt.scatter(cycles, capacities, alpha=0.6, label='실험 데이터')
plt.plot(cycles, true_capacity, 'r--', alpha=0.7, label='실제 곡선')
plt.xlabel('사이클 수')
plt.ylabel('잔존 용량 비율')
plt.title('배터리 용량 열화 데이터 (45°C)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f'데이터 포인트 수: {len(cycles)}')
print(f'사이클 범위: {cycles[0]} ~ {cycles[-1]}')
print(f'용량 범위: {capacities.min():.3f} ~ {capacities.max():.3f}')

### 2.1 모델 피팅

In [ ]:
# 모델 생성 및 피팅
model = CapacityDegradationModel()

print('모델 피팅 중...')
result = model.fit(cycles, temperatures, capacities)

print('\n=== 피팅 결과 ===')
print(f'RMSE: {result.rmse:.6f}')
print(f'R²: {result.r_squared:.6f}')
print('\n파라미터:')
for key, value in result.to_dict().items():
    if key not in ['rmse', 'r_squared']:
        print(f'  {key}: {value:.6f}')

### 2.2 예측 및 시각화

In [ ]:
# 더 많은 포인트에서 예측
predict_cycles = np.linspace(0, 1500, 200)
predict_temps = np.full_like(predict_cycles, 318.15)
predicted_capacity = model.predict(predict_cycles, predict_temps)

# 시각화
plt.figure(figsize=(12, 6))
plt.scatter(cycles, capacities, alpha=0.6, s=50, label='실험 데이터', color='blue')
plt.plot(predict_cycles, predicted_capacity, 'r-', linewidth=2, label='피팅된 모델')
plt.axhline(y=0.8, color='green', linestyle='--', alpha=0.7, label='EOL 기준 (80%)')
plt.xlabel('사이클 수', fontsize=12)
plt.ylabel('잔존 용량 비율', fontsize=12)
plt.title('용량 열화 모델 피팅 결과 (45°C)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 1500)
plt.ylim(0.5, 1.05)
plt.show()

# 잔차 플롯
fitted_values = model.predict(cycles, temperatures)
residuals = capacities - fitted_values

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.scatter(cycles, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.7)
plt.xlabel('사이클 수')
plt.ylabel('잔차')
plt.title('잔차 플롯')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(residuals, bins=15, alpha=0.7, edgecolor='black')
plt.xlabel('잔차')
plt.ylabel('빈도')
plt.title('잔차 분포')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2.3 EOL (End-of-Life) 예측

In [ ]:
# EOL 사이클 예측 (80% 용량 기준)
eol_cycle = model.predict_cycle_to_eol(
    temperature=318.15,  # 45°C
    eol_threshold=0.8,
    max_cycles=5000
)

print(f'\n=== EOL 예측 ===')
print(f'시험 온도: 45°C')
print(f'EOL 기준: 80% 잔존 용량')
print(f'예측 EOL 사이클: {eol_cycle:,} 사이클')

# EOL 지점 표시
eol_capacity = model.predict(np.array([eol_cycle]), np.array([318.15]))[0]
print(f'EOL 시점 예측 용량: {eol_capacity:.3f}')

## 3. 승인 수명 예측 (ApprovalLifePredictor)

가속 시험 데이터를 실사용 조건으로 환산하여 승인 수명을 예측합니다.

### Arrhenius 가속 계수
```
AF = exp(Ea/R * (1/T_target - 1/T_test))
```

- Ea: 활성화 에너지 (J/mol)
- R: 기체 상수 (8.314 J/mol·K)
- T_test: 시험 온도 (K)
- T_target: 실사용 온도 (K)

In [ ]:
# 승인 수명 예측기 생성
predictor = ApprovalLifePredictor(
    test_temperature=318.15,      # 45°C 시험 온도
    target_temperature=298.15,    # 25°C 실사용 온도
    cycles_per_year=365,          # 하루 1사이클 가정
    required_years=8.0            # 8년 승인 요구사항
)

print('=== 승인 수명 예측 설정 ===')
print(f'시험 온도: 45°C')
print(f'실사용 온도: 25°C')
print(f'연간 사이클: 365 (하루 1회)')
print(f'요구 수명: 8년')

### 3.1 가속 계수 계산 및 민감도 분석

In [ ]:
# 가속 계수 계산
af = predictor.calculate_arrhenius_factor(activation_energy=50000)  # 50 kJ/mol
print(f'\n가속 계수 (Ea=50kJ/mol): {af:.2f}배')
print(f'의미: 45°C에서 1사이클 = 25°C에서 {af:.2f}사이클')

# 민감도 분석
sensitivity_df = predictor.sensitivity_analysis(
    activation_energy_range=(40000, 80000),
    n_points=20
)

# 시각화
plt.figure(figsize=(10, 5))
plt.plot(sensitivity_df['activation_energy_kJ'], 
         sensitivity_df['acceleration_factor'], 
         'b-', linewidth=2, marker='o')
plt.axvline(x=50, color='r', linestyle='--', alpha=0.7, label='기본값 (50 kJ/mol)')
plt.xlabel('활성화 에너지 (kJ/mol)', fontsize=12)
plt.ylabel('가속 계수', fontsize=12)
plt.title('활성화 에너지에 따른 가속 계수 민감도', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print('\n민감도 분석 결과:')
print(sensitivity_df.head())

### 3.2 승인 수명 예측 수행

In [ ]:
# 피팅 및 예측
approval_result = predictor.fit_and_predict(
    cycles=cycles,
    temperatures=temperatures,
    capacities=capacities,
    eol_threshold=0.8,
    activation_energy=50000
)

print('\n=== 승인 수명 예측 결과 ===')
print(f'시험 EOL 사이클: {approval_result.test_cycles:,} 사이클')
print(f'가속 계수: {approval_result.acceleration_factor:.2f}배')
print(f'실사용 환산 사이클: {approval_result.equivalent_real_cycles:,} 사이클')
print(f'예상 수명: {approval_result.predicted_years:.2f}년')
print(f'요구 수명: {approval_result.required_years}년')
print(f'\n승인 기준 충족: {"✓ 통과" if approval_result.meets_requirement else "✗ 미달"}')

# 결과를 딕셔너리로 변환
result_dict = approval_result.to_dict()
print('\n전체 결과:')
for key, value in result_dict.items():
    if isinstance(value, float):
        print(f'  {key}: {value:.4f}')
    else:
        print(f'  {key}: {value}')

### 3.3 다양한 온도 조건에서의 수명 비교

In [ ]:
# 여러 온도에서 수명 예측
target_temps = [15, 20, 25, 30, 35]  # °C
results = []

for temp_c in target_temps:
    temp_predictor = ApprovalLifePredictor(
        test_temperature=318.15,
        target_temperature=273.15 + temp_c,
        cycles_per_year=365,
        required_years=8.0
    )
    
    result = temp_predictor.fit_and_predict(
        cycles=cycles,
        temperatures=temperatures,
        capacities=capacities,
        activation_energy=50000
    )
    
    results.append({
        'temperature_C': temp_c,
        'acceleration_factor': result.acceleration_factor,
        'predicted_years': result.predicted_years,
        'meets_requirement': result.meets_requirement
    })

# 결과를 DataFrame으로 변환
temp_comparison_df = pd.DataFrame(results)

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 가속 계수
ax1.plot(temp_comparison_df['temperature_C'], 
         temp_comparison_df['acceleration_factor'], 
         'b-o', linewidth=2, markersize=8)
ax1.set_xlabel('실사용 온도 (°C)', fontsize=12)
ax1.set_ylabel('가속 계수', fontsize=12)
ax1.set_title('온도별 가속 계수', fontsize=13)
ax1.grid(True, alpha=0.3)

# 예상 수명
colors = ['green' if x else 'red' for x in temp_comparison_df['meets_requirement']]
ax2.bar(temp_comparison_df['temperature_C'].astype(str), 
        temp_comparison_df['predicted_years'],
        color=colors, alpha=0.7, edgecolor='black')
ax2.axhline(y=8, color='red', linestyle='--', linewidth=2, label='요구 수명 (8년)')
ax2.set_xlabel('실사용 온도 (°C)', fontsize=12)
ax2.set_ylabel('예상 수명 (년)', fontsize=12)
ax2.set_title('온도별 예상 수명', fontsize=13)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('\n온도별 예측 결과:')
print(temp_comparison_df.to_string(index=False))

## 4. 요약 및 결론

이 노트북에서는 `battery_analysis_v2` 모듈의 핵심 기능을 시연했습니다:

1. **용량 열화 모델링**: 경험적 모델을 사용하여 배터리 용량 열화를 정확하게 예측
2. **EOL 예측**: 특정 온도 조건에서 배터리가 80% 용량에 도달하는 사이클 수 계산
3. **승인 수명 예측**: Arrhenius 방정식을 사용하여 가속 시험 결과를 실사용 조건으로 환산
4. **민감도 분석**: 활성화 에너지 및 온도 조건에 따른 영향 분석

### 추가 기능

모듈에는 다음과 같은 추가 기능도 포함되어 있습니다:
- dV/dQ 분석 (`battery_analysis_v2.core.dvdq_analysis`)
- Full-cell 시뮬레이션 (`FullCellSimulator`)
- EU 수명 예측 (`EULifePredictor`)

자세한 내용은 각 모듈의 docstring을 참고하세요.

---

## 참고사항

> **Note**: 이 예제는 시뮬레이션 데이터를 사용합니다. 실제 배터리 시험 데이터로 대체하여 사용하세요.

> **Important**: numba 최적화가 제거되어 순수 Python 구현만 사용합니다. 대용량 데이터 처리 시 성능을 고려하세요.

### 데이터 형식

실제 데이터 사용 시 다음 형식을 따르세요:
- `cycles`: 1D numpy array (사이클 수)
- `temperatures`: 1D numpy array (온도, Kelvin 단위)
- `capacities`: 1D numpy array (잔존 용량 비율, 0-1)